In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
def to_sequence(data, seq_size=10):
    x, y = [], []

    for i in range(len(data)-seq_size):
        window = data[i:(i+seq_size)].values
        after_window = data[i+seq_size]
        x.append(window)
        y.append(after_window)

    return np.array(x), np.array(y)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,40):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'/content/Patient_{j}.csv')
  print(j)

  for i in range(len(df)):
    if np.isnan(df.at[i,'SensorGLU']):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
      # Create a SVR model
      model = SVR(kernel='linear',gamma='auto', C=1.0, epsilon = 0.1)
      # Train the model
      model.fit(X_train, y_train)
      # Make Predictions
      df.at[i,'SensorGLU']= model.predict((df.loc[i-10:i-1, "SensorGLU"].values).reshape(1,10))


  # Model Assessing
  pred = model.predict(X_test)
  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [12]:
df

,ReadingDateTime,SensorGLU
0,2000-02-10 14:21:00,67.000000
1,2000-02-10 14:26:00,76.000000
2,2000-02-10 14:31:00,86.000000
3,2000-02-10 14:36:00,98.000000
4,2000-02-10 14:41:00,102.000000
...,...,...
343,2000-02-11 18:56:00,82.309258
344,2000-02-11 19:01:00,54.000000
345,2000-02-11 19:06:00,52.000000
346,2000-02-11 19:11:00,55.000000


In [13]:
print(vertical_MAE)

[ 4.65354673  2.59932453  3.98237446  8.34701242  5.25450313 24.89843991
  8.23566228  7.88690279  5.94120436  4.95713497  8.51386067  4.22860181
  7.85129366  2.89945212  2.95673271 25.65330694  5.23616744 11.79244277
  6.20417593  4.91007717  3.80208178 14.32441957  6.03968132  5.45679005
  6.36588976  9.35152242  5.68477035  6.16084216  5.30269675  6.24164013
  6.23583125  3.64272566 11.63978422  1.63267156  4.96065653  5.08928227
  5.63254842  9.91494125  9.13413307]


In [14]:
np.median(vertical_MAE)

5.941204355601628

In [15]:
len(vertical_MAE)

39

In [16]:
vertical_RMSE

array([ 7.08022781,  4.10385914,  5.94430501, 16.06577847,  8.27405295,
       54.45253113, 14.90459978, 16.98781773, 16.04300069,  8.32583842,
       13.86990419,  7.47782314, 15.01289053,  6.66268107,  5.04732471,
       42.64708161,  9.1581916 , 21.57883953, 10.78044914,  8.85780761,
        7.68003688, 30.92251926, 13.76244643, 10.71767145,  9.79218156,
       18.82261972,  9.08115589, 11.92865521,  9.4891688 , 11.33121903,
       10.28139056,  6.28587269, 19.69766595,  2.64516377,  9.70436544,
       10.57370769,  9.326716  , 19.0882497 , 16.83934708])

In [17]:
np.median(vertical_RMSE)

10.573707693734198

In [18]:
model.score(X_test,y_test)

0.9478714896304599

In [19]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'/content/Patient_result_SVR1_40_part1.csv', index=False)